# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
M. Zhang  ->  M. Zhang  |  ['M. Zhang']
J. Li  ->  J. Li  |  ['J. Li']
J. Shi  ->  J. Shi  |  ['J. Shi']
H. Jiang  ->  H. Jiang  |  ['H. Jiang']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
J. Shi  ->  J. Shi  |  ['J. Shi']
P. Griveaud  ->  P. Griveaud  |  ['P. Griveaud']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']


Arxiv has 71 new papers today
          8 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2508.06608


extracting tarball to tmp_2508.06608... done.
Retrieving document from  https://arxiv.org/e-print/2508.06892


extracting tarball to tmp_2508.06892...

 done.
Retrieving document from  https://arxiv.org/e-print/2508.06936


...

P. Griveaud  ->  P. Griveaud  |  ['P. Griveaud']


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure Figures/RK4_lin_traj
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure Figures/RK4_lin_traj as tmp_2508.07859/./Figures/RK4_lin_traj.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure Figures/RK4_exppi2_traj
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure Figures/RK4_exppi2_traj as tmp_2508.07859/./Figures/RK4_exppi2_traj.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))

Found 15 bibliographic references in tmp_2508.07859/ITLetter_openjournal.bbl.
Retrieving document from  https://arxiv.org/e-print/2508.07988


extracting tarball to tmp_2508.07988...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.07859-b31b1b.svg)](https://arxiv.org/abs/2508.07859) | **The reflex instability: exponential growth of a large-scale $m=1$ mode in astrophysical discs**  |
|| A. Crida, et al. -- incl., <mark>P. Griveaud</mark> |
|*Appeared on*| *2025-08-12*|
|*Comments*| *Published in the Open Journal of Astrophysics*|
|**Abstract**|            We report the finding of a linear, non-axisymmetric, global instability in gas discs around stars, which may be relevant to other astrophysical discs. It takes the form of an $m=1$ mode that grows in the disc density distribution while the star-barycentre distance rises exponentially with a characteristic timescale that is orders of magnitude longer than the orbital period. We present results of hydrodynamical simulations with various codes and numerical methods, using either barycentric or stellocentric reference frames, with or without the disc's self gravity: all simulations consistently show an unstable mode growing exponentially. The instability disappears if, and only if, the reflex motion of the star due to the disc's asymmetry is not taken into account in the simulations. For this reason we refer to this instability as the reflex instability. We identify a feedback loop as a possible origin, whereby the acceleration of the star excites the eccentricity of the disc, yielding an $m=1$ mode in the density distribution which, in turn, pulls the star. The growth timescale of the instability decreases with increasing disc mass and is a few hundred orbits for disc-to-star mass ratios of a few percent. If truly physical, and not due to a numerical artifact that would be common to all the codes we have employed, the reflex instability could have a dramatic impact on protoplanetary discs evolution and planetary formation.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.06608-b31b1b.svg)](https://arxiv.org/abs/2508.06608) | **Second public data release of the FIRE-2 cosmological zoom-in simulations of galaxy formation**  |
|| A. Wetzel, et al. -- incl., <mark>K. El-Badry</mark> |
|*Appeared on*| *2025-08-12*|
|*Comments*| *12 pages. Data available at this http URL*|
|**Abstract**|            We describe the second data release (DR2) of the FIRE-2 cosmological zoom-in simulations of galaxy formation, from the Feedback In Realistic Environments (FIRE) project, available at this http URL. DR2 includes all snapshots for most simulations, starting at z ~ 99, with all snapshot time spacings <~ 25 Myr. The Core suite -- comprising 14 Milky Way-mass galaxies, 5 SMC/LMC-mass galaxies, and 4 lower-mass galaxies -- includes 601 snapshots to z = 0. For the Core suite, we also release resimulations with physics variations: (1) dark-matter-only versions; (2) a modified ultraviolet background with later reionization at z = 7.8; (3) magnetohydrodynamics, anisotropic conduction, and viscosity in gas; and (4) a model for cosmic-ray injection, transport, and feedback (assuming a constant diffusion coefficient). The Massive Halo suite now includes 8 massive galaxies with 278 snapshots to z = 1. The High Redshift suite includes 34 simulations: in addition to the 22 simulations run to z = 5, we now include 12 additional simulations run to z = 7 and z = 9. Most simulations include catalogs of (sub)halos and galaxies at all available snapshots, and most Core simulations to z = 0 include full halo merger trees.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.06892-b31b1b.svg)](https://arxiv.org/abs/2508.06892) | **Large Model Driven Solar Activity AI Forecaster: A Scalable Dual Data-Model Framework**  |
|| J. Wang, et al. -- incl., <mark>M. Zhang</mark> |
|*Appeared on*| *2025-08-12*|
|*Comments*| **|
|**Abstract**|            Solar activity drives space weather, affecting Earth's magnetosphere and technological infrastructure, which makes accurate solar flare forecasting critical. Current space weather models under-utilize multi-modal solar data, lack iterative enhancement via expert knowledge, and rely heavily on human forecasters under the Observation-Orientation-Decision-Action (OODA) paradigm. Here we present the "Solar Activity AI Forecaster", a scalable dual data-model driven framework built on foundational models, integrating expert knowledge to autonomously replicate human forecasting tasks with quantifiable outputs. It is implemented in the OODA paradigm and comprises three modules: a Situational Perception Module that generates daily solar situation awareness maps by integrating multi-modal observations; In-Depth Analysis Tools that characterize key solar features (active regions, coronal holes, filaments); and a Flare Prediction Module that forecasts strong flares for the full solar disk and active regions. Executed within a few minutes, the model outperforms or matches human forecasters in generalization across multi-source data, forecast accuracy, and operational efficiency. This work establishes a new paradigm for AI-based space weather forecasting, demonstrating AI's potential to enhance forecast accuracy and efficiency, and paving the way for autonomous operational forecasting systems.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.06936-b31b1b.svg)](https://arxiv.org/abs/2508.06936) | **Pre-perihelion radio observations of comet 12P/Pons-Brooks with Tianma radio telescope**  |
|| <mark>J. Li</mark>, et al. -- incl., <mark>J. Shi</mark> |
|*Appeared on*| *2025-08-12*|
|*Comments*| *9 pages, 5 figures. Accepted for publication in A&A*|
|**Abstract**|            {The multiple outburst events of comet 12P/Pons-Brooks during its 2024 apparition offer a unique window into highly-active volatile releasing processes not observable during quiescent periods. We performed radio observations of comet 12P/Pons-Brooks with the Tianma-65m radio telescope, targeting the OH and NH$_3$ inversion lines at 18-cm and 1.3-cm, respectively. By monitoring 12P at different heliocentric distances on its inbound journey, we aim to provide insights into the comet's volatile composition and outburst behavior. Four observations were carried out between December 2023 and March 2024 when the comet was approaching the Sun from 2.22 AU to 1.18 AU. We conducted 18-cm OH lines observations on 4 single days using the cryogenically cooled receiver system of the telescope to derive $\rm H_{2}O$ production rate. During 12P's outburst on December 14, we also conducted observations targeting the $\rm NH_{3}$ emission. OH 18-cm lines were clearly detected with a signal-to-noise ratio of $\sim$4$\sigma$ (peak intensity). A tentative detection of $\rm NH_{3}$ was made at the $\sim$$3\sigma$ level during the outburst phase, but the detection needs to be further verified. Our observations provide information on the outgassing behavior of 12P/Pons-Brooks during its 2024 apparition. The water production rate of 12P, derived from the 18-cm OH lines is consistent with measurements obtained in other works. The possible detection of $\rm NH_{3}$ during an outburst suggests possible connections between subsurface volatile reservoir and the outburst mechanism. These results could further our understanding of the composition and activity of Halley-type comets.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.07065-b31b1b.svg)](https://arxiv.org/abs/2508.07065) | **Reconstruction of Solar EUV Irradiance Using CaII K Images and SOHO/SEM Data with Bayesian Deep Learning and Uncertainty Quantification**  |
|| <mark>H. Jiang</mark>, et al. |
|*Appeared on*| *2025-08-12*|
|*Comments*| *18 pages, 10 figures*|
|**Abstract**|            Solar extreme ultraviolet (EUV) irradiance plays a crucial role in heating the Earth's ionosphere, thermosphere, and mesosphere, affecting atmospheric dynamics over varying time scales. Although significant effort has been spent studying short-term EUV variations from solar transient events, there is little work to explore the long-term evolution of the EUV flux over multiple solar cycles. Continuous EUV flux measurements have only been available since 1995, leaving significant gaps in earlier data. In this study, we propose a Bayesian deep learning model, named SEMNet, to fill the gaps. We validate our approach by applying SEMNet to construct SOHO/SEM EUV flux measurements in the period between 1998 and 2014 using CaII K images from the Precision Solar Photometric Telescope. We then extend SEMNet through transfer learning to reconstruct solar EUV irradiance in the period between 1950 and 1960 using CaII K images from the Kodaikanal Solar Observatory. Experimental results show that SEMNet provides reliable predictions along with uncertainty bounds, demonstrating the feasibility of CaII K images as a robust proxy for long-term EUV fluxes. These findings contribute to a better understanding of solar influences on Earth's climate over extended periods.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.07161-b31b1b.svg)](https://arxiv.org/abs/2508.07161) | **The impact of organic hazes and graphite on the observation of CO2-rich sub-Neptune atmospheres**  |
|| H. Li, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-08-12*|
|*Comments*| *12 pages, 5 figures, accepted at APJL*|
|**Abstract**|            Many sub-Neptune and super-Earth exoplanets are expected to develop metal-enriched atmospheres due to atmospheric loss processes such as photoevaporation or core-powered mass loss. Thermochemical equilibrium calculations predict that at high metallicity and a temperature range of 300-700 K, CO2 becomes the dominant carbon species, and graphite may be the thermodynamically favored condensate under low-pressure conditions. Building on prior laboratory findings that such environments yield organic haze rather than graphite, we measured the transmittance spectra of organic haze analogues and graphite samples, and computed their optical constants across the measured wavelength range from 0.4 to 25 {\mu}m. The organic haze exhibits strong vibrational absorption bands, notably at 3.0, 4.5, and 6.0 {\mu}m, while graphite shows featureless broadband absorption. The derived optical constants of haze and graphite provide the first dataset for organic haze analogues formed in CO2-rich atmospheres and offer improved applicability over prior graphite data derived from bulk reflectance or ellipsometry. We implemented these optical constants into the Virga and PICASO cloud and radiative transfer models to simulate transit spectra for GJ 1214b. The synthetic spectra with organic hazes reproduce the muted spectral features in the NIR observed by Hubble and general trends observed by JWST for GJ 1214b, while graphite models yield flat spectra across the observed wavelengths. This suggests haze features may serve as observational markers of carbon-rich atmospheres, whereas graphite's opacity could lead to radius overestimation, offering a possible explanation for super-puff exoplanets. Our work supplies essential optical to infrared data for interpreting observations of CO2-rich exoplanet atmospheres.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.07599-b31b1b.svg)](https://arxiv.org/abs/2508.07599) | **Identification of OB associations using the LAMOST-Gaia OB star sample**  |
|| Z. Liu, et al. -- incl., <mark>J. Shi</mark> |
|*Appeared on*| *2025-08-12*|
|*Comments*| *17 pages, 15 figures, 2 tables, accepted in ApJS*|
|**Abstract**|            OB associations, as an intermediate stage between Galactic clusters and field stars, play an important role in understanding the star formation process, early stellar evolution, and Galactic evolution. In this work, we construct a large sample of OB stars with 6D phase space parameters ($l, b, d, V_{\rm los}, pmra ,pmdec$) by combining the distances from Bailer-Jones et al. (2021), radial velocities derived from low-resolution spectra of the Large Sky Area Multi-Object Fiber Spectroscopic Telescope (LAMOST), and proper motions from the \textit{Gaia} Data Release 3 (DR3). This sample includes 19,933 OB stars, most of which are located within 6\,kpc of the Sun. Using 6D phase space parameters and friends-of-friends clustering algorithm, we identify 67 OB associations and 112 OB association candidates, among them, 49 OB associations and 107 OB association candidates are newly identified. The Galactic rotation curve derived using 67 OB association members is relatively flat in the range of Galactocentric distances 7$<$$R$$<$13\,kpc. The angular rotation velocity at solar Galactocentric distance of $R_\odot$ =8.34\,kpc is $\Omega_0$ = 29.05$\pm$0.55\,km\,s$^{-1}$\,kpc$^{-1}$. The spatial distribution of the 67 OB associations indicates that they are mainly located at low Galactic latitudes and near spiral arms of the Milky Way. Additionally, we estimate the velocity dispersions and sizes of these 67 OB associations. Our results show that the velocity dispersions decrease as Galactocentric distances increase, while their sizes increase as Galactocentric distances increase.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.07988-b31b1b.svg)](https://arxiv.org/abs/2508.07988) | **A Data-constrained Magnetohydrodynamic Simulation of Successive X-class Flares in Solar Active Region 13842. II. Dynamics of the Solar Eruption Associated with the X9.0 Solar Flare**  |
|| K. Matsumoto, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-08-12*|
|*Comments*| *10 Figures. Accepted to ApJ*|
|**Abstract**|            Active region NOAA 13842 produced two successive solar flares: an X7.1-class flare on October 1, 2024, and an X9.0-class flare on October 3, 2024. This study continues our previous simulation work that successfully reproduced the X7.1-class solar flare (Matsumoto et al. 2025). In this study, we performed a data-constrained magnetohydrodynamic (MHD) simulation using the nonlinear force-free field (NLFFF) as the initial condition to investigate the X9.0-class solar flare. The NLFFF showed the sheared field lines, resulting in the tether-cutting reconnection, the magnetic flux ropes (MFRs), and eventually led to eruption. The magnetic reconnection during the pre-eruption phase plays a critical role in accelerating the subsequent eruption, which is driven by torus instability and magnetic reconnection. Furthermore, our simulation results are consistent with several observational features associated with the X9.0 flare. This simulation could reproduce diverse phenomena associated with the X9.0 flare, including the tether-cutting reconnection, the flare ribbons and the flare loops, the transverse field enhancement, and the remote brightening away from the flare ribbons. However, the initial trigger, magnetic flux emergence, was inferred from observations rather than explicitly modeled, and future comprehensive simulations should incorporate this mechanism directly.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2508.07859/./Figures/fig2a.png', 'tmp_2508.07859/./Figures/fig2b_new2.png', 'tmp_2508.07859/./Figures/RK4_lin_traj.png', 'tmp_2508.07859/./Figures/RK4_exppi2_traj.png', 'tmp_2508.07859/./Figures/RK4_exppi8_traj.png', 'tmp_2508.07859/./Figures/RK4_e_evol_tight.png', 'tmp_2508.07859/./Figures/fig1a.png', 'tmp_2508.07859/./Figures/fig1b.png', 'tmp_2508.07859/./Figures/fig1c.png']
copying  tmp_2508.07859/./Figures/fig2a.png to _build/html/
copying  tmp_2508.07859/./Figures/fig2b_new2.png to _build/html/
copying  tmp_2508.07859/./Figures/RK4_lin_traj.png to _build/html/
copying  tmp_2508.07859/./Figures/RK4_exppi2_traj.png to _build/html/
copying  tmp_2508.07859/./Figures/RK4_exppi8_traj.png to _build/html/
copying  tmp_2508.07859/./Figures/RK4_e_evol_tight.png to _build/html/
copying  tmp_2508.07859/./Figures/fig1a.png to _build/html/
copying  tmp_2508.07859/./Figures/fig1b.png to _build/html/
copying  tmp_2508.07859/./Figures/fig1c.png to _build/html/
exported in  _bui

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\myeqref}[1]{Eq. (\ref{#1})}$
$\newcommand{\figref}[1]{Fig.~\ref{#1}}$
$\newcommand{\Figref}[1]{Figure~\ref{#1}}$
$\newcommand{\ITdd}{\texttt{ITdd}}$
$\newcommand{\fargo}{{\sc fargo}}$
$\newcommand{\fargotwodoned}{{\sc fargo-2d1d}}$
$\newcommand{\fargoadsg}{{\sc fargo-adsg}}$
$\newcommand{\fargothreed}{{\sc fargo3d}}$
$\newcommand{\inewcommandix}{{\sc inewcommandix}}$
$\newcommand{\phantomcode}{{\sc phantom}}$
$\newcommand{\sbt}{ \begin{picture}(-1,1)(1,-3)\circle*{3}\end{picture} }$
$\newcommand{\hm}[2]{\textcolor{green}{#1}\textcolor{limegreen}{\st{#2}}}$
$\newcommand{\el}[1]{\textcolor{magenta}{#1}}$
$\newcommand{\jeff}[2]{\textcolor{teal}{#1} \textcolor{teal}{\st{#2}}}$
$\newcommand{\ac}[2]{\textcolor{blue}{#1} \textcolor{blue}{\st{#2}}}$
$\newcommand{\cb}[1]{\textcolor{NavyBlue}{#1}}$
$\newcommand{\mnr}[2]{\textcolor{teal}{#1}}$</div>



<div id="title">

# The reflex instability: \\exponential growth of a large-scale $m=1$ mode in astrophysical discs$\vspace{-16pt}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2508.07859-b31b1b.svg)](https://arxiv.org/abs/2508.07859)<mark>Appeared on: 2025-08-12</mark> -  _Published in the Open Journal of Astrophysics_

</div>
<div id="authors">

A. Crida, et al. -- incl., <mark>P. Griveaud</mark>

</div>
<div id="abstract">

**Abstract:** We report the finding of a linear, non-axisymmetric, global instability in gas discs around stars, which may be relevant to other astrophysical discs. It takes the form of an $m=1$ mode that grows in the disc density distribution while the star-barycentre distance rises exponentially with a characteristic timescale that is orders of magnitude longer than the orbital period.  We present results of hydrodynamical simulations with various codes and numerical methods, using either barycentric or stellocentric reference frames, with or without the disc's self gravity: all simulations consistently show an unstable mode growing exponentially.  The instability disappears if, and only if, the reflex motion of the star due to the disc's asymmetry is not taken into account in the simulations. For this reason we refer to this instability as the $*reflex instability*$ .  We identify a feedback loop as a possible origin, whereby the acceleration of the star excites the eccentricity of the disc, yielding an $m=1$ mode in the density distribution which, in turn, pulls the star.  The growth timescale of the instability decreases with increasing disc mass and is a few hundred orbits for disc-to-star mass ratios of a few percent. If truly physical, and not due to a numerical artifact that would be common to all the codes we have employed, the reflex instability could have a dramatic impact on protoplanetary discs evolution and planetary formation.

</div>

<div id="div_fig1">

<img src="tmp_2508.07859/./Figures/fig2a.png" alt="Fig4.1" width="50%"/><img src="tmp_2508.07859/./Figures/fig2b_new2.png" alt="Fig4.2" width="50%"/>

**Figure 4. -** Time evolution of the radial distance between the star and the centre-of-mass of the star-disc system for simulations using our reference setup, obtained for different codes and by varying numerical parameters (see caption and text). (*fig:com_tlogr_numerics*)

</div>
<div id="div_fig2">

<img src="tmp_2508.07859/./Figures/RK4_lin_traj.png" alt="Fig2.1" width="25%"/><img src="tmp_2508.07859/./Figures/RK4_exppi2_traj.png" alt="Fig2.2" width="25%"/><img src="tmp_2508.07859/./Figures/RK4_exppi8_traj.png" alt="Fig2.3" width="25%"/><img src="tmp_2508.07859/./Figures/RK4_e_evol_tight.png" alt="Fig2.4" width="25%"/>

**Figure 2. -** Orbital evolution of a particle feeling an acceleration $\vec{a}=-(1/r^2)\vec{u}_r+\vec{a}_c$. Top left panel : $\vec{a}_c=0.004 \vec{u}_y$. Top right : $\vec{a}_c = 0.02 \mathcal{R}_{\pi/2}(\vec{P})$. Bottom left panel : $\vec{a}_c = 0.02 \mathcal{R}_{\pi/8}(\vec{P})$, where $\mathcal{R}_\phi$ is the rotation by an angle $\phi$ in the $x-y$ plane. Bottom right panel : evolution of the eccentricity in the three simulations (same color).
     (*fig:RK4*)

</div>
<div id="div_fig3">

<img src="tmp_2508.07859/./Figures/fig1a.png" alt="Fig3.1" width="33%"/><img src="tmp_2508.07859/./Figures/fig1b.png" alt="Fig3.2" width="33%"/><img src="tmp_2508.07859/./Figures/fig1c.png" alt="Fig3.3" width="33%"/>

**Figure 3. -** Results of our reference setup for the $\fargo$twodoned simulation which uses a barycentric frame. The left panel shows the position of the star in the barycentric frame. The open square symbols mark the star's position in the middle and right panels, and the arrows display the acceleration exerted by the disc on the star (arbitrary magnitude). The middle and right panels display the perturbation of the surface density of the disc relative to its azimuthally-averaged initial density, at two times separated by about half a pattern period. The solid curve shows the inner edge of the computational grid, the dashed curves show the edges of the inner and outer wave-damping zones. (*fig:bary*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2508.07859"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

132  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
